In [2]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
import numpy as np
from matplotlib import pyplot as plt
import torch
from sparse_causal_model_learner_rl.sacred_gin_tune.sacred_wrapper import load_config_files
from sparse_causal_model_learner_rl.learners.rl_learner import CausalModelLearnerRL
from sparse_causal_model_learner_rl.config import Config
from keychest.features_xy import arr_to_dict, dict_to_arr, obs_features_handcoded
from keychest.gofa_model import manual_model_features
from sparse_causal_model_learner_rl.trainable.gumbel_switch import LearnableSwitch, WithInputSwitch
from tqdm.auto import tqdm
from torch import nn
import gin

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
p = LearnableSwitch((10, ))

In [4]:
list(p.named_parameters())

[('logits',
  Parameter containing:
  tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True))]

In [5]:
loss = torch.mean(p.sample_mask(method='gumbel'))

In [6]:
p(torch.randn(4, 10))

tensor([[ 0.5868, -1.4454,  2.3385,  0.0000, -0.0000,  0.0000,  0.1970,  0.0000,
          0.4822,  0.3403],
        [ 1.2387, -0.0397, -0.3968,  0.0000, -0.0000, -0.0000, -0.2353,  0.0000,
          0.4775,  0.8199],
        [-0.2817, -1.3355, -0.8617, -0.0000, -0.0000, -0.0000, -2.1774,  0.0000,
         -1.1071, -0.2839],
        [-1.1118, -0.0599, -2.0636, -0.0000,  0.0000,  0.0000, -0.4872,  0.0000,
         -0.4715, -1.6643]], grad_fn=<MulBackward0>)

In [7]:
p.logits.grad = None

In [8]:
#loss.backward()
loss = torch.mean(p(torch.randn(4, 10)))

In [9]:
loss.backward()

In [10]:
p.logits.grad

tensor([[ 0.0104,  0.0021, -0.0012,  0.0071,  0.0043,  0.0137, -0.0072, -0.0085,
         -0.0006,  0.0059],
        [-0.0104, -0.0021,  0.0012, -0.0071, -0.0043, -0.0137,  0.0072,  0.0085,
          0.0006, -0.0059]])

In [11]:
p.sparsify_me()

[('proba_on',
  tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000], grad_fn=<SelectBackward>))]

In [12]:
p.softmaxed()

tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], grad_fn=<SelectBackward>)

In [13]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__()
        self.z = nn.Linear(10, 1)
    def forward(self, x):
        return self.z(x)

In [14]:
ws = WithInputSwitch(MyModel, input_shape=(10,))

In [15]:
list(ws.named_parameters())

[('switch.logits',
  Parameter containing:
  tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True)),
 ('model.z.weight',
  Parameter containing:
  tensor([[ 0.0395,  0.1642,  0.1554,  0.2993,  0.2281, -0.2956, -0.2262,  0.2894,
            0.0011,  0.1516]], requires_grad=True)),
 ('model.z.bias',
  Parameter containing:
  tensor([-0.2651], requires_grad=True))]

In [16]:
ws.sparsify_me()

[('proba_on',
  tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000], grad_fn=<SelectBackward>))]

In [26]:
ws(torch.randn(5, 10))

tensor([[-0.3655],
        [-0.0679],
        [ 0.1606],
        [-0.6880],
        [-0.2407]], grad_fn=<AddmmBackward>)

In [27]:
ws.switch(torch.randn(5, 10))

tensor([[ 0.0296,  0.0000,  0.0000, -0.7063, -0.0000, -1.2354, -0.4303, -0.0000,
         -1.2880,  0.1100],
        [-0.0073,  0.0000, -0.0000,  1.0901, -0.0000, -2.1113, -0.3207, -0.0000,
         -0.6150, -1.6298],
        [ 0.7556, -0.0000,  0.0000, -0.0869,  0.0000,  0.0900, -1.0956, -0.0000,
          0.4013,  0.6033],
        [-1.4899, -0.0000,  0.0000,  0.8348, -0.0000, -0.0973,  0.8588,  0.0000,
         -2.3550, -2.4604],
        [ 0.0823, -0.0000, -0.0000,  0.7506,  0.0000, -0.5795, -0.7919, -0.0000,
         -0.8441,  0.8194]], grad_fn=<MulBackward0>)